##### NGUYỄN TRẦN LÂM - 20016701 

In [37]:
# import thư viện 
import pandas as pd
import re
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from joblib import dump, load
from sklearn.feature_extraction.text import HashingVectorizer

##### Bài 1:

Dữ liệu được thu thập từ trang báo `vnexpress.net` <br> gồm các Label:
- Chứng khoán
- Doanh nghiệp
- Ebank 
- Bóng đá 
- Khoa học 

In [4]:
df = pd.read_csv("Data.csv", encoding='utf-8')
df.head(10)

,title,content,category
0,Học đầu tư chứng khoán thế nào với người mới b...,Tôi mới tìm hiểu thị trường chứng khoán gần đâ...,Chứng khoán
1,VN-Index lên mức cao nhất 9 tháng,"VN-Index chốt phiên tại 1.129,38 điểm, tăng hơ...",Chứng khoán
2,DIG | Dòng tiền đổ mạnh vào cổ phiếu DIG,"Sau phiên ATO, thị giá DIG nhảy vọt lên vùng 2...",Chứng khoán
3,Chứng khoán tăng phiên thứ hai liên tiếp,Chỉ số đại diện cho sàn TP HCM đóng cửa tại 1....,Chứng khoán
4,Chứng khoán lội ngược dòng,"Mở cửa phiên hôm nay, chỉ số đại diện sàn HoSE...",Chứng khoán
5,Cổ phiếu Novaland gần chạm sàn,"Như dự báo của giới phân tích, thị trường đối ...",Chứng khoán
6,Tiền vào chứng khoán tăng trở lại,"Sau khi chạm đáy vào cuối tháng 4, thanh khoản...",Chứng khoán
7,Chứng khoán tuần này đối mặt nhịp điều chỉnh,Thị trường vừa ghi nhận một tuần giao dịch run...,Chứng khoán
8,Chứng khoán đảo chiều giảm,"Sáng nay, chỉ số đại diện cho sàn TP HCM tăng ...",Chứng khoán
9,'Cổ phiếu chứng khoán kỳ vọng khởi sắc từ nay ...,"Theo bà Nguyễn Ngọc Linh, Giám đốc tự doanh Ch...",Chứng khoán


1.  Tiền xử lý dữ liệu với Beautiful Soup, re,... 

In [5]:
def processcing_text(text):
    tweet = text
    #lower
    tweet = tweet.lower()
    #convert any url link to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet)
    #convert any @Username to AT_USER
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet)

    #Remove not alphanumeric symbols white spaces
    tweet = re.sub(r'[^\w]',' ', tweet)
    #Removes # hashtag in front of a word
    tweet = re.sub(r'#([\w]+)', r'\1', tweet)
    tweet = re.sub(r'#([^\s]+)',r'\1',tweet)
    #remove :( or :)
    tweet = tweet.replace(':)','')
    tweet = tweet.replace(':(','')
    #remove numbers
    tweet = ''.join([i for i in tweet if not i.isdigit()])
    #remove multiple exclamation
    tweet = re.sub(r'(!)\1+', ' ', tweet)
    #remove multiple question marks
    tweet = re.sub(r'(\?)\1+','', tweet)
    #remove multistop
    tweet = re.sub(r'(\.)\1+','', tweet)
    #Remove additional whitespace
    tweet = re.sub(r'[\s]+',' ', tweet)
    tweet = re.sub(r'[\n]+',' ', tweet)
    row = tweet
    return row

df['content']=df['content'].apply(processcing_text)
df.head(1)['content'].values[0]

'tôi mới tìm hiểu thị trường chứng khoán gần đây mọi thứ quá phức tạp nên tôi muốn học kỹ lĩnh vực này sau đó sẽ bỏ ít vốn ra thực hành chấp nhận lỗ rồi mới tính chuyện đầu tư lâu dài tôi muốn được chuyên gia tư vấn giúp lộ trình tham khảo từ việc học tập về chứng khoán cho đến giai đoạn thử thực hành và chính thức tham gia thị trường ở mỗi giai đoạn tôi cần tập trung hoặc chú ý những điều gì trần quang ngọc nhà đầu tư đang theo dõi thị trường tại một sàn giao dịch chứng khoán ở quận tp hcm ảnh quỳnh trần chuyên gia tư vấn chứng khoán nói chung và cổ phiếu nói riêng là kênh đầu tư rất hấp dẫn thu hút được nhiều nhà đầu tư ở mọi lứa tuổi đây là kênh đầu tư có nhiều lợi ích trong đó nổi bật nhất là tính thanh khoản có thể bắt đầu từ số vốn vừa và nhỏ ở đây tôi sẽ tạm hiểu nhu cầu của bạn là đầu tư vào cổ phiếu và mong muốn được biết thêm về lộ trình và cách học đầu tư cổ phiếu để tham gia thị trường thành công và hiệu quả bạn đã xác định đúng trong việc đầu tư vào kiến thức nhằm giúp bản

2.  Tách từ (Tokenize) sử dụng thư viện pyvi hay underthesea

In [6]:
from underthesea import word_tokenize


def toke(text):
    t=word_tokenize(text, format='text')
    # t=word_tokenize(t)
    return t

df['content']=df['content'].apply(toke)
df['content'].head(1).values[0]

'tôi mới tìm_hiểu thị_trường_chứng_khoán gần đây mọi thứ quá phức_tạp nên tôi muốn học_kỹ lĩnh_vực này sau đó sẽ bỏ ít vốn ra thực_hành chấp_nhận lỗ rồi mới tính_chuyện đầu_tư lâu_dài tôi muốn được chuyên_gia tư_vấn giúp lộ_trình tham_khảo từ việc học_tập về chứng_khoán cho đến giai_đoạn thử thực_hành và chính_thức tham_gia thị_trường ở mỗi giai_đoạn tôi cần tập_trung hoặc chú_ý những điều gì trần_quang ngọc nhà_đầu_tư đang theo_dõi thị_trường tại một sàn giao_dịch chứng_khoán ở quận tp hcm ảnh quỳnh_trần chuyên_gia tư_vấn chứng_khoán nói_chung và cổ_phiếu nói_riêng là kênh đầu_tư rất hấp_dẫn thu_hút được nhiều nhà_đầu_tư ở mọi lứa tuổi đây là kênh đầu_tư có nhiều lợi_ích trong đó nổi_bật nhất là tính thanh_khoản có_thể bắt_đầu từ số vốn vừa và nhỏ ở đây tôi sẽ tạm hiểu nhu_cầu của bạn là đầu_tư vào cổ_phiếu và mong_muốn được biết thêm về lộ_trình và cách học đầu_tư cổ_phiếu để tham_gia thị_trường thành_công và hiệu_quả bạn đã xác_định đúng trong việc đầu_tư vào kiến_thức nhằm giúp bản

3. Trích xuất đặc trưng TF-IDF bằng thư viện sklearn

In [15]:
corpus = df['content'].values
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(corpus)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['category'].values)
X.toarray().shape, y.shape

((49, 2670), (49,))

4. Đánh giá bộ dữ liệu với giải thuật KNN bằng phương pháp 5-Fold (k-fold)


In [19]:
label_encoder.inverse_transform([0,1,2])

array(['Chứng khoán', 'Doanh nghiệp', 'Ebank'], dtype=object)

In [21]:
knn = KNeighborsClassifier(n_neighbors=2)  

kf = KFold(n_splits=5, shuffle=True, random_state=42)

accuracies = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    knn.fit(X_train, y_train)

    y_pred = knn.predict(X_val)

    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)
print(f'Average Accuracy: {average_accuracy}')

Average Accuracy: 0.8


5. Huấn luyện dữ liệu cho bài toán phân loại văn bản với tỷ lệ dữ liệu 8:2 (8 phần train, 2 phần test) sử dụng đặc trưng TF-IDF và 2 giải thuật bayes và SVM

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nb_classifier = GaussianNB()
nb_classifier.fit(X_train.toarray() , y_train)
nb_predictions = nb_classifier.predict(X_test.toarray())

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)
svm_predictions = svm_classifier.predict(X_test)

6. Tính độ đo F1 score

In [30]:
print('F1 score of Naive Bayes: ', f1_score(y_test, nb_predictions, average='micro'))
print('F1 score of SVM: ', f1_score(y_test, svm_predictions, average='micro'))

F1 score of Naive Bayes:  0.7
F1 score of SVM:  0.9


SVM có F1 score cao hơn so với Naive Bayes, cho thấy SVM có khả năng phân loại tốt hơn trên tập kiểm tra.

7. Tính độ đo Accurary

In [31]:
print('Accuracy score of Naive Bayes: ', accuracy_score(y_test, nb_predictions))
print('Accuracy score of SVM: ', accuracy_score(y_test, svm_predictions))

Accuracy score of Naive Bayes:  0.7
Accuracy score of SVM:  0.9


Accuracy của SVM cao hơn so với Naive Bayes, cho thấy SVM dự đoán chính xác hơn trên tập kiểm tra.

8. Tính độ đo Confusion Matrix

In [32]:
print('Confusion matrix of Naive Bayes: \n', confusion_matrix(y_test, nb_predictions)) 
print('Confusion matrix of SVM: \n', confusion_matrix(y_test, svm_predictions))

Confusion matrix of Naive Bayes: 
 [[0 1 0]
 [0 4 2]
 [0 0 3]]
Confusion matrix of SVM: 
 [[0 1 0]
 [0 6 0]
 [0 0 3]]


SVM có nhiều trường hợp True Positives hơn và không có False Negatives nào, trong khi Naive Bayes có một số False Negatives.

9. So sánh kết quả các độ đo 6,7,8 với 2 giải thuật học máy ở trên


Dựa trên các độ đo F1 score, Accuracy và Confusion Matrix, mô hình SVM hiện có hiệu suất tốt hơn so với mô hình Naive Bayes trên tập kiểm tra cụ thể này.

10. Lưu model với giải thuật đạt kết quả tốt nhất

In [34]:
dump(nb_classifier, 'nb_classifier.pkl')
dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

- Test:

In [35]:
loaded_nb_classifier = load('nb_classifier.pkl')
loaded_tfidf_vectorizer = load('tfidf_vectorizer.pkl')

In [36]:
def predict_text_to_label(text):
    text = processcing_text(text)
    text = toke(text)
    text = loaded_tfidf_vectorizer.transform([text]).toarray()
    label = loaded_nb_classifier.predict(text)
    label_name = label_encoder.inverse_transform(label)
    return label_name[0]

text_test='Một vụ tai nạn giao thông thảm khốc xảy ra ở tân bình'
predict_text_to_label(text_test)

'Doanh nghiệp'

##### Bài 2: 

In [38]:
corpus = [
 ' Hôm_nay tôi đi_học',
 ' Hôm_nay tôi đi_học ở trường',
 ' Hôm_nay tôi nghỉ ở nhà',
 ' Hôm_nay tôi có đi_học không?',
]
vectorizer = HashingVectorizer(n_features=2**5)
X = vectorizer.fit_transform(corpus)
print(X.shape)

(4, 32)


Mỗi từ trong câu sẽ được băm thành một số theo một hàm băm cố định. Ví dụ, từ "Hôm_nay" có thể được băm thành 1, "tôi" có thể được băm thành 2, "đi_học" có thể được băm thành 3, và cứ tiếp tục.

Đối với mỗi câu, các số băm này sẽ được sử dụng để xác định các features trong vectơ biểu diễn. Mỗi câu sẽ có một vectơ số có kích thước cố định, trong trường hợp này là 16 chiều.

Các features trong vectơ được xác định dựa trên giá trị của các số băm. Ví dụ, nếu trong câu có từ "đi_học" được băm thành 3, thì có thể có một feature tại vị trí 3 trong vectơ.

Kết quả của HashingVectorizer là một ma trận, trong đó mỗi hàng tương ứng với một câu trong corpus và mỗi cột tương ứng với một feature. Số trong mỗi ô của ma trận thể hiện mức độ xuất hiện của feature tương ứng trong câu tương ứng.

##### Bài 3: Sử dụng HashVectorizer thay cho đặc trưng TF-IDF ở bài 1

In [40]:
hash_vec = HashingVectorizer(n_features=2**14)
corpus = df['content'].values
X = hash_vec.fit_transform(corpus)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['category'].values)
X.toarray().shape, y.shape

((49, 16384), (49,))

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nb_classifier = GaussianNB()
nb_classifier.fit(X_train.toarray() , y_train)
nb_predictions = nb_classifier.predict(X_test.toarray())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_classifier = SVC(kernel='linear')

svm_classifier.fit(X_train, y_train)

svm_predictions = svm_classifier.predict(X_test)


print('F1 score of Naive Bayes: ', f1_score(y_test, nb_predictions, average='micro'))
print('F1 score of SVM: ', f1_score(y_test, svm_predictions, average='micro'))
print('Accuracy score of Naive Bayes: ', accuracy_score(y_test, nb_predictions))
print('Accuracy score of SVM: ', accuracy_score(y_test, svm_predictions))
print('Confusion matrix of Naive Bayes: \n', confusion_matrix(y_test, nb_predictions)) 
print('Confusion matrix of SVM: \n', confusion_matrix(y_test, svm_predictions))

F1 score of Naive Bayes:  0.7
F1 score of SVM:  0.7
Accuracy score of Naive Bayes:  0.7
Accuracy score of SVM:  0.7
Confusion matrix of Naive Bayes: 
 [[0 1 0]
 [0 4 2]
 [0 0 3]]
Confusion matrix of SVM: 
 [[0 1 0]
 [1 4 1]
 [0 0 3]]
